# VectorStore

1. ChromaDB
2. FAISS
3. Qdrant-client

## Import Libraries

In [1]:
import fitz
import numpy as np
import re
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
import os

## Lecacy Code

In [2]:
# --- OpenAI 클라이언트 설정 ---
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 헬퍼 함수 (기존 노트북과 동일) ---

def extract_text_from_pdf(pdf_path):
    """PDF 파일에서 텍스트를 추출합니다."""
    mypdf = fitz.open(pdf_path)
    all_text = ""
    for page in mypdf:
        all_text += page.get_text("text")
    return all_text

def chunk_text(text, n, overlap):
    """주어진 텍스트를 n자 단위로, 지정된 수의 문자가 겹치도록 분할합니다."""
    chunks = []
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])
    return chunks

def generate_questions(text_chunk, num_questions=5, model="gpt-4o-mini"):
    """주어진 텍스트 청크로부터 관련 질문들을 생성합니다."""
    system_prompt = (
        "당신은 텍스트로부터 관련 질문을 생성하는 전문가입니다. "
        "제공된 텍스트를 바탕으로 그 내용에만 근거한 간결한 질문들을 생성하세요."
    )
    user_prompt = f"다음 텍스트를 기반으로, 해당 텍스트만으로 답할 수 있는 서로 다른 질문 {num_questions}개를 생성하세요:\n\n{text_chunk}\n\n응답은 번호가 매겨진 질문 리스트 형식으로만 작성하고, 그 외 부가 설명은 하지 마세요."

    response = client.chat.completions.create(
        model=model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    questions_text = response.choices[0].message.content.strip()
    questions = [re.sub(r'^\d+\.\s*', '', line.strip()) for line in questions_text.split('\n') if line.strip()]
    return [q for q in questions if q.endswith('?')]


def create_embeddings(text, model="text-embedding-3-small"):
    """지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다."""
    response = client.embeddings.create(model=model, input=text)
    return response

## 1. Chroma (ChromaDB)

In [3]:
%pip install -q chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.5 which is incompatible.
grpcio-tools 1.73.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
grpcio-health-checking 1.73.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
grpcio-status 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import chromadb

def process_document_with_chroma(
    pdf_path, 
    chunk_size=1000, 
    chunk_overlap=200, 
    questions_per_chunk=3
):
    """문서를 처리하여 Chroma 컬렉션에 저장합니다."""
    # 1. 텍스트 추출 및 청크 분할
    extracted_text = extract_text_from_pdf(pdf_path)
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(text_chunks)}개의 텍스트 청크 생성 완료.")

    # 2. Chroma 클라이언트 및 컬렉션 설정
    chroma_client = chromadb.Client() # 인-메모리 클라이언트
    collection = chroma_client.get_or_create_collection(name="document_store_chroma")

    # 3. 각 청크 처리 및 Chroma에 추가
    all_embeddings = []
    all_documents = []
    all_metadatas = []
    all_ids = []

    for i, chunk in enumerate(tqdm(text_chunks, desc="Chroma 처리 중")):
        # 청크 임베딩
        chunk_embedding = create_embeddings(chunk).data[0].embedding
        all_embeddings.append(chunk_embedding)
        all_documents.append(chunk)
        all_metadatas.append({"type": "chunk", "index": i})
        all_ids.append(f"chunk_{i}")

        # 질문 생성 및 임베딩
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        if questions:
            question_embeddings = create_embeddings(questions).data
            for j, (q, q_emb) in enumerate(zip(questions, question_embeddings)):
                all_embeddings.append(q_emb.embedding)
                all_documents.append(q)
                all_metadatas.append({"type": "question", "chunk_index": i, "original_chunk": chunk})
                all_ids.append(f"question_{i}_{j}")

    # 모든 데이터를 한 번에 추가
    collection.add(
        embeddings=all_embeddings,
        documents=all_documents,
        metadatas=all_metadatas,
        ids=all_ids
    )

    print(f"Chroma에 총 {collection.count()}개의 항목 저장 완료.")
    return collection

# --- 실행 및 검색 예시 ---
pdf_path = "../dataset/AI_Understanding.pdf"
chroma_collection = process_document_with_chroma(pdf_path)

# 검색 쿼리
query_text = "인공지능의 윤리적 문제는 무엇인가?"
query_embedding = create_embeddings(query_text).data[0].embedding

# Chroma에서 유사도 검색 수행
results = chroma_collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

print("\n--- Chroma 검색 결과 ---")
for i, (doc, meta, dist) in enumerate(zip(results['documents'][0], results['metadatas'][0], results['distances'][0])):
    print(f"Rank {i+1} (Distance: {dist:.4f}):")
    print(f"  Type: {meta.get('type')}")
    print(f"  Text: {doc[:200]}...") # 텍스트가 길 경우 일부만 출력
    print("-" * 20)

총 21개의 텍스트 청크 생성 완료.


Chroma 처리 중: 100%|██████████| 21/21 [00:54<00:00,  2.59s/it]


Chroma에 총 84개의 항목 저장 완료.

--- Chroma 검색 결과 ---
Rank 1 (Distance: 0.6437):
  Type: question
  Text: 자율 무기 시스템에 AI를 사용할 때의 윤리적 우려는 무엇인가요?...
--------------------
Rank 2 (Distance: 0.7951):
  Type: chunk
  Text:  등에 
사용됩니다. AI 알고리즘은 대규모 데이터 세트를 분석하여 패턴을 파악하고, 시장 동향을 
예측하고, 금융 프로세스를 자동화할 수 있습니다. 
교통편 
AI는 자율주행차, 교통 최적화 시스템, 물류 관리의 발전으로 교통 분야에 혁신을 일으키고 
있습니다. 자율 주행 차량은 AI를 사용하여 주변 환경을 인식하고, 주행 결정을 내리고, 
안전하게 주행...
--------------------
Rank 3 (Distance: 0.8244):
  Type: chunk
  Text:  심각한 윤리적, 보안적 우려가 제기될 수 있습니다. AI 
기반 무기와 관련된 위험을 해결하기 위해 국제적인 논의와 규제가 필요합니다. 
 
5장: 인공 지능의 미래 
AI의 미래는 다양한 영역에서 지속적인 발전과 폭넓은 도입으로 특징지어질 것입니다. 
주요 트렌드와 개발 분야는 다음과 같습니다: 
설명 가능한 AI(XAI) 
설명 가능한 AI(XAI)는...
--------------------
Rank 4 (Distance: 0.8851):
  Type: chunk
  Text: 동화하고 위협 탐지 정확도를 
개선하며 전반적인 사이버 보안 태세를 강화할 수 있습니다. 
 
4장: AI의 윤리적, 사회적 의미 
AI의 급속한 발전과 보급은 윤리적, 사회적으로 심각한 우려를 불러일으킵니다. 이러한 
우려에는 다음이 포함됩니다: 
편견과 공정성 
AI 시스템은 데이터에 존재하는 편견을 계승하고 증폭시켜 불공정하거나 차별적인 결과를 
초래...
--------------------
Rank 5 (Distance: 

## 2. FAISS (Facebook AI Similarity Search)

In [5]:
%pip install -q faiss-cpu 
# 또는 GPU 버전: faiss-gpu

Note: you may need to restart the kernel to use updated packages.


In [6]:
import faiss

def process_document_with_faiss(
    pdf_path, 
    chunk_size=1000, 
    chunk_overlap=200, 
    questions_per_chunk=3
):
    """문서를 처리하여 FAISS 인덱스와 메타데이터 리스트를 생성합니다."""
    # 1. 텍스트 추출 및 청크 분할
    extracted_text = extract_text_from_pdf(pdf_path)
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(text_chunks)}개의 텍스트 청크 생성 완료.")

    # 2. FAISS 인덱스 및 메타데이터 저장소 초기화
    embedding_dim = 1536  # OpenAI 'text-embedding-3-small' 모델의 차원
    index = faiss.IndexFlatIP(embedding_dim) # 코사인 유사도를 위한 내적(IP) 인덱스

    # FAISS는 벡터 외 정보를 저장하지 않으므로 별도 리스트 관리
    metadata_store = []

    for i, chunk in enumerate(tqdm(text_chunks, desc="FAISS 처리 중")):
        # 청크 임베딩 및 정규화
        chunk_embedding = create_embeddings(chunk).data[0].embedding
        chunk_vector = np.array(chunk_embedding, dtype='float32')
        faiss.normalize_L2(chunk_vector.reshape(1, -1))

        # 인덱스와 메타데이터 저장소에 추가
        index.add(chunk_vector.reshape(1, -1))
        metadata_store.append({"text": chunk, "type": "chunk", "index": i})

        # 질문 생성 및 임베딩
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        if questions:
            question_embeddings_data = create_embeddings(questions).data
            q_vectors = np.array([item.embedding for item in question_embeddings_data], dtype='float32')
            faiss.normalize_L2(q_vectors)

            index.add(q_vectors)
            for j, q in enumerate(questions):
                metadata_store.append({"text": q, "type": "question", "chunk_index": i, "original_chunk": chunk})

    print(f"FAISS에 총 {index.ntotal}개의 항목 저장 완료.")
    return index, metadata_store

# --- 실행 및 검색 예시 ---
pdf_path = "../dataset/AI_Understanding.pdf"
faiss_index, faiss_metadata = process_document_with_faiss(pdf_path)

# 검색 쿼리
query_text = "인공지능의 윤리적 문제는 무엇인가?"
query_embedding = create_embeddings(query_text).data[0].embedding
query_vector = np.array(query_embedding, dtype='float32').reshape(1, -1)
faiss.normalize_L2(query_vector) # 쿼리 벡터도 정규화

# FAISS에서 유사도 검색 수행
k = 5
distances, indices = faiss_index.search(query_vector, k)

print("\n--- FAISS 검색 결과 ---")
for i in range(k):
    idx = indices[0][i]
    dist = distances[0][i]
    meta = faiss_metadata[idx]

    print(f"Rank {i+1} (Similarity: {dist:.4f}):")
    print(f"  Type: {meta.get('type')}")
    print(f"  Text: {meta.get('text')[:200]}...")
    print("-" * 20)

총 21개의 텍스트 청크 생성 완료.


FAISS 처리 중: 100%|██████████| 21/21 [00:50<00:00,  2.42s/it]


FAISS에 총 84개의 항목 저장 완료.

--- FAISS 검색 결과 ---
Rank 1 (Similarity: 0.6725):
  Type: question
  Text: 인공 지능(AI)의 정의는 무엇인가요?...
--------------------
Rank 2 (Similarity: 0.6595):
  Type: question
  Text: 자율 무기 시스템에 AI를 사용할 때 발생할 수 있는 윤리적 우려는 무엇인가요?...
--------------------
Rank 3 (Similarity: 0.6506):
  Type: question
  Text: AI의 윤리적 우려 중 개인정보 보호와 데이터 보안이 중요한 이유는 무엇인가요?...
--------------------
Rank 4 (Similarity: 0.6500):
  Type: question
  Text: 윤리적 AI의 주요 원칙에는 어떤 것들이 포함되나요?...
--------------------
Rank 5 (Similarity: 0.6024):
  Type: chunk
  Text:  등에 
사용됩니다. AI 알고리즘은 대규모 데이터 세트를 분석하여 패턴을 파악하고, 시장 동향을 
예측하고, 금융 프로세스를 자동화할 수 있습니다. 
교통편 
AI는 자율주행차, 교통 최적화 시스템, 물류 관리의 발전으로 교통 분야에 혁신을 일으키고 
있습니다. 자율 주행 차량은 AI를 사용하여 주변 환경을 인식하고, 주행 결정을 내리고, 
안전하게 주행...
--------------------


## 3. Qdrant

In [12]:
%pip install -q qdrant-client

Note: you may need to restart the kernel to use updated packages.


In [7]:
from qdrant_client import QdrantClient, models
import uuid

def process_document_with_qdrant(
    pdf_path, 
    chunk_size=1000, 
    chunk_overlap=200, 
    questions_per_chunk=3
):
    """문서를 처리하여 Qdrant 컬렉션에 저장합니다."""
    # 1. 텍스트 추출 및 청크 분할
    extracted_text = extract_text_from_pdf(pdf_path)
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(text_chunks)}개의 텍스트 청크 생성 완료.")

    # 2. Qdrant 클라이언트 및 컬렉션 설정
    client = QdrantClient(":memory:") # 인-메모리 클라이언트
    collection_name = "document_store_qdrant"
    
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=1536,  # OpenAI 'text-embedding-3-small' 모델의 차원
            distance=models.Distance.COSINE
        )
    )

    # 3. 각 청크 처리 및 Qdrant에 추가 (배치 처리)
    points_to_upsert = []
    for i, chunk in enumerate(tqdm(text_chunks, desc="Qdrant 처리 중")):
        # 청크 임베딩
        chunk_embedding = create_embeddings(chunk).data[0].embedding
        
        # Qdrant에 저장할 Point 생성
        points_to_upsert.append(
            models.PointStruct(
                id=str(uuid.uuid4()), # 고유 ID 생성
                vector=chunk_embedding,
                payload={"text": chunk, "type": "chunk", "index": i}
            )
        )

        # 질문 생성 및 임베딩
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        if questions:
            question_embeddings_data = create_embeddings(questions).data
            for q, q_emb in zip(questions, question_embeddings_data):
                points_to_upsert.append(
                    models.PointStruct(
                        id=str(uuid.uuid4()),
                        vector=q_emb.embedding,
                        payload={"text": q, "type": "question", "chunk_index": i, "original_chunk": chunk}
                    )
                )

    # 모든 Point를 한 번에 업로드 (Upsert)
    client.upsert(
        collection_name=collection_name,
        points=points_to_upsert,
        wait=True # 작업 완료 대기
    )
    
    count_result = client.count(collection_name=collection_name, exact=True)
    print(f"Qdrant에 총 {count_result.count}개의 항목 저장 완료.")
    return client, collection_name

# --- 실행 및 검색 예시 ---
pdf_path = "../dataset/AI_Understanding.pdf"
qdrant_client, collection_name = process_document_with_qdrant(pdf_path)

# 검색 쿼리
query_text = "인공지능의 윤리적 문제는 무엇인가?"
query_embedding = create_embeddings(query_text).data[0].embedding

# Qdrant에서 유사도 검색 수행
search_result = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_embedding,
    limit=5 # 상위 5개 결과 반환
)

print("\n--- Qdrant 검색 결과 ---")
for i, hit in enumerate(search_result):
    print(f"Rank {i+1} (Score: {hit.score:.4f}):")
    print(f"  Type: {hit.payload.get('type')}")
    print(f"  Text: {hit.payload.get('text')[:200]}...")
    print("-" * 20)

/tmp/ipykernel_3155103/1847974152.py:20: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


총 21개의 텍스트 청크 생성 완료.


Qdrant 처리 중: 100%|██████████| 21/21 [00:48<00:00,  2.31s/it]


Qdrant에 총 84개의 항목 저장 완료.

--- Qdrant 검색 결과 ---
Rank 1 (Score: 0.6675):
  Type: question
  Text: 윤리적 AI 원칙의 주요 내용은 무엇입니까?...
--------------------
Rank 2 (Score: 0.6352):
  Type: question
  Text: 인공 지능의 정의는 무엇인가요?...
--------------------
Rank 3 (Score: 0.6025):
  Type: chunk
  Text:  등에 
사용됩니다. AI 알고리즘은 대규모 데이터 세트를 분석하여 패턴을 파악하고, 시장 동향을 
예측하고, 금융 프로세스를 자동화할 수 있습니다. 
교통편 
AI는 자율주행차, 교통 최적화 시스템, 물류 관리의 발전으로 교통 분야에 혁신을 일으키고 
있습니다. 자율 주행 차량은 AI를 사용하여 주변 환경을 인식하고, 주행 결정을 내리고, 
안전하게 주행...
--------------------
Rank 4 (Score: 0.5887):
  Type: chunk
  Text:  심각한 윤리적, 보안적 우려가 제기될 수 있습니다. AI 
기반 무기와 관련된 위험을 해결하기 위해 국제적인 논의와 규제가 필요합니다. 
 
5장: 인공 지능의 미래 
AI의 미래는 다양한 영역에서 지속적인 발전과 폭넓은 도입으로 특징지어질 것입니다. 
주요 트렌드와 개발 분야는 다음과 같습니다: 
설명 가능한 AI(XAI) 
설명 가능한 AI(XAI)는...
--------------------
Rank 5 (Score: 0.5730):
  Type: question
  Text: AI 기반 무기와 관련된 윤리적, 보안적 우려를 해결하기 위해 필요한 조치는 무엇인가요?...
--------------------


/tmp/ipykernel_3155103/1847974152.py:76: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(
